In [ ]:
!pip install -q langchain langchain-google-genai google-generativeai faiss-cpu rank_bm25 ragas datasets

In [7]:
!pip install -q numpy langchain google-generativeai faiss-cpu rank_bm25 ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [10]:
import numpy as np
# from langchain.embeddings import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from rank_bm25 import BM25Okapi
from typing import List, Tuple

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from datasets import Dataset

In [12]:
%pip install -Uq sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00


In [13]:
from sentence_transformers import SentenceTransformer

######not working

In [11]:
class GeminiRAG:
    def __init__(self, documents: List[str], embedding_model=None, knoweledge_base=None):
        self.documents = documents
        # Use Gemini embedding model via LangChain (GoogleGenerativeAIEmbeddings)
        self.embedding_model = sentence-transformers/all-MiniLM-L6-v2
        # Semantic Index (FAISS)
        self.faiss_index = FAISS.from_texts(documents, self.embedding_model)
        # BM25 Index
        self.bm25 = BM25Okapi([doc.split() for doc in documents])
        """
        Initialize the GeminiRAG instance.

        Args:
            knowledge_base: Optional knowledge base object to perform RAG.
        """
        self.knowledge_base = knoweledge_base

    def semantic_search(self, query: str, top_k: int = 5) -> List[Tuple[str, float]]:
        docs_and_scores = self.faiss_index.similarity_search_with_score(query, k=top_k)
        return [(doc.page_content, score) for doc, score in docs_and_scores]

    def bm25_search(self, query: str, top_k: int = 5) -> List[Tuple[str, float]]:
        scores = self.bm25.get_scores(query.split())
        top_indices = np.argsort(scores)[::-1][:top_k]
        return [(self.documents[i], scores[i]) for i in top_indices]

    def hybrid_search(self, query: str, top_k: int = 5, alpha: float = 0.5) -> List[Tuple[str, float]]:
        sem_results = dict(self.semantic_search(query, top_k=top_k*2))
        bm25_results = dict(self.bm25_search(query, top_k=top_k*2))
        sem_max = max(sem_results.values(), default=1)
        bm25_max = max(bm25_results.values(), default=1)
        sem_results_norm = {k: v / sem_max for k, v in sem_results.items()}
        bm25_results_norm = {k: v / bm25_max for k, v in bm25_results.items()}
        all_docs = set(sem_results) | set(bm25_results)
        hybrid_scores = {
            doc: alpha * sem_results_norm.get(doc, 0) + (1 - alpha) * bm25_results_norm.get(doc, 0)
            for doc in all_docs
        }
        ranked = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return ranked

    def hyde(self, question: str, model=None) -> str:
        """
        Generate a hypothetical answer (HyDE) to the question to use as a retrieval query.
        Uses Gemini via LangChain.
        """
        # Use LangChain's Gemini LLM wrapper (GoogleGenerativeAI)
        from langchain.llms import GoogleGenerativeAI
        llm = model or GoogleGenerativeAI(model="models/gemini-pro", temperature=0)
        hypothetical_answer = llm(question)
        return hypothetical_answer

    def hyde_hybrid_search(self, question: str, top_k: int = 5, alpha: float = 0.5, model=None) -> List[Tuple[str, float]]:
        hypothetical_answer = self.hyde(question, model=model)
        return self.hybrid_search(hypothetical_answer, top_k=top_k, alpha=alpha)

    def chat(self, prompt: str) -> Tuple[str, List[str]]:
        """
        Generate a response to the given prompt using RAG.

        Args:
            prompt (str): The user input prompt.

        Returns:
            Tuple[str, List[str]]: The generated response and the retrieved context.
        """
        # Example implementation (replace with actual logic)
        context = []
        if self.knowledge_base:
            context = self.knowledge_base.retrieve(prompt)
            # Replace with actual LLM call using context
            response = f"Context: {context}\nResponse: This is a generated answer based on the context."
        else:
             # Replace with actual LLM call without context
             response = "Response: This is a generated answer without context."

        return response, context

    def evaluate_ragas(self, query: str, answer: str, context: List[str]):
        """
        Evaluate RAG metrics for a given query, answer, and context.

        Args:
            query (str): The user query.
            answer (str): The generated answer.
            context (List[str]): The retrieved context.

        Returns:
            dict: A dictionary containing the Ragas metrics.
        """
        data = {
            "question": [query],
            "answer": [answer],
            "contexts": [context]
        }
        dataset = Dataset.from_dict(data)

        result = evaluate(
            dataset,
            metrics=[
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy,
          ] ,
        )
        return result.to_pandas().iloc[0].to_dict()

    def evaluate_dataset(self, dataset: Dataset):
        """
        Evaluate RAG metrics for a dataset of queries, answers, and contexts.

        Args:
            dataset (Dataset): The dataset containing queries, answers, and contexts.

        Returns:
            pandas.DataFrame: A DataFrame containing the Ragas metrics for each sample in the dataset.
        """
        return evaluate(
            dataset,
            metrics=[
              context_precision,
              context_recall,
              faithfulness,
              answer_relevancy,
          ],
        ).to_pandas()

In [ ]:
# Assuming you have a GeminiRAG instance named rag_system
# And you have a query, generated answer, and retrieved context

query = "What is the capital of France?"
answer = "Paris is the capital of France."
context = ["France is a country in Europe. Its capital is Paris."]

# Evaluate a single sample
ragas_metrics = rag_system.evaluate_ragas(query, answer, context)
print(ragas_metrics)

# Assuming you have a dataset in the format of a datasets.Dataset object
# with columns "question", "answer", and "contexts"

# Evaluate a dataset
dataset_ragas_metrics = rag_system.evaluate_dataset(dataset)
print(dataset_ragas_metrics)

######working

In [21]:
from sentence_transformers import SentenceTransformer
import numpy as np
from langchain.vectorstores import FAISS
from rank_bm25 import BM25Okapi
from typing import List, Tuple
from langchain.schema import Document # Import Document

class SentenceTransformerRAG:
    def __init__(self, documents: List[str], model_name: str = 'sentence-transformers/all-MiniLM-L6-v2'):
        self.documents = documents
        # Use SentenceTransformer model
        self.embedding_model = SentenceTransformer(model_name)
        # Create embeddings for the documents
        document_embeddings = self.embedding_model.encode(documents)
        # Semantic Index (FAISS)
        self.faiss_index = FAISS.from_embeddings([(documents[i], document_embeddings[i]) for i in range(len(documents))], self.embedding_model)
        # BM25 Index
        self.bm25 = BM25Okapi([doc.split() for doc in documents])

    def semantic_search(self, query: str, top_k: int = 5) -> List[Tuple[str, float]]:
        query_embedding = self.embedding_model.encode(query)
        # similarity_search_by_vector returns a list of Document objects, not tuples.
        docs_with_scores = self.faiss_index.similarity_search_by_vector(query_embedding, k=top_k)
        # Extract page_content and score from Document objects
        return [(doc.page_content, doc.metadata.get('score', 0.0)) for doc in docs_with_scores]


    def bm25_search(self, query: str, top_k: int = 5) -> List[Tuple[str, float]]:
        scores = self.bm25.get_scores(query.split())
        top_indices = np.argsort(scores)[::-1][:top_k]
        return [(self.documents[i], scores[i]) for i in top_indices]

    def hybrid_search(self, query: str, top_k: int = 5, alpha: float = 0.5) -> List[Tuple[str, float]]:
        # Call semantic_search and bm25_search, which now return the correct format
        sem_results = dict(self.semantic_search(query, top_k=top_k*2))
        bm25_results = dict(self.bm25_search(query, top_k=top_k*2))

        # Normalize scores (simple min-max normalization)
        sem_scores = np.array(list(sem_results.values()))
        bm25_scores = np.array(list(bm25_results.values()))

        sem_min, sem_max = sem_scores.min(), sem_scores.max()
        bm25_min, bm25_max = bm25_scores.min(), bm25_scores.max()

        sem_results_norm = {k: (v - sem_min) / (sem_max - sem_min) if (sem_max - sem_min) != 0 else 0 for k, v in sem_results.items()}
        bm25_results_norm = {k: (v - bm25_min) / (bm25_max - bm25_min) if (bm25_max - bm25_min) != 0 else 0 for k, v in bm25_results.items()}


        all_docs = set(sem_results.keys()) | set(bm25_results.keys())
        hybrid_scores = {
            doc: alpha * sem_results_norm.get(doc, 0) + (1 - alpha) * bm25_results_norm.get(doc, 0)
            for doc in all_docs
        }
        ranked = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return ranked

    def hyde(self, question: str, model=None) -> str:
        """
        Generate a hypothetical answer (HyDE) to the question to use as a retrieval query.
        Uses a provided LLM (e.g., Gemini via LangChain).
        """
        if model is None:
             raise ValueError("An LLM model must be provided for HyDE generation.")

        hypothetical_answer = model.invoke(question) # Assuming the model has an invoke method
        return hypothetical_answer

    def hyde_hybrid_search(self, question: str, top_k: int = 5, alpha: float = 0.5, model=None) -> List[Tuple[str, float]]:
        hypothetical_answer = self.hyde(question, model=model)
        return self.hybrid_search(hypothetical_answer, top_k=top_k, alpha=alpha)

    def chat(self, prompt: str, llm_model=None) -> Tuple[str, List[str]]:
        """
        Generate a response to the given prompt using RAG.

        Args:
            prompt (str): The user input prompt.
            llm_model: The language model to use for generation.

        Returns:
            Tuple[str, List[str]]: The generated response and the retrieved context.
        """
        if llm_model is None:
             raise ValueError("An LLM model must be provided for chat generation.")

        # Example implementation using hybrid search for context
        context_docs = self.hybrid_search(prompt, top_k=3) # Retrieve top 3 relevant documents
        context = [doc for doc, score in context_docs]

        # Combine context and prompt for the LLM
        context_text = "\n".join(context)
        formatted_prompt = f"Context: {context_text}\n\nQuestion: {prompt}\n\nAnswer:"

        response = llm_model.invoke(formatted_prompt) # Assuming the model has an invoke method

        return response, context

    # Evaluation methods (evaluate_ragas and evaluate_dataset) can be added here
    # if needed, similar to the GeminiRAG class.

In [22]:
# Sample documents
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is a rapidly evolving field.",
    "Natural language processing is a subfield of AI.",
    "The sun is a star at the center of our solar system.",
    "The moon is a natural satellite of the Earth."
]

# Create an instance of the SentenceTransformerRAG class
rag_system_st = SentenceTransformerRAG(documents)

# Example queries
query_ai = "What is AI?"
query_space = "Tell me about the moon."
query_nlp = "What is NLP?"

In [23]:
# Perform semantic search
print(f"Semantic search results for '{query_ai}':")
semantic_results = rag_system_st.semantic_search(query_ai)
for doc, score in semantic_results:
    print(f"  Score: {score:.4f}, Document: {doc}")

print(f"\nSemantic search results for '{query_space}':")
semantic_results = rag_system_st.semantic_search(query_space)
for doc, score in semantic_results:
    print(f"  Score: {score:.4f}, Document: {doc}")

Semantic search results for 'What is AI?':
  Score: 0.0000, Document: Artificial intelligence is a rapidly evolving field.
  Score: 0.0000, Document: Natural language processing is a subfield of AI.
  Score: 0.0000, Document: The moon is a natural satellite of the Earth.
  Score: 0.0000, Document: The sun is a star at the center of our solar system.
  Score: 0.0000, Document: The quick brown fox jumps over the lazy dog.

Semantic search results for 'Tell me about the moon.':
  Score: 0.0000, Document: The moon is a natural satellite of the Earth.
  Score: 0.0000, Document: The sun is a star at the center of our solar system.
  Score: 0.0000, Document: Artificial intelligence is a rapidly evolving field.
  Score: 0.0000, Document: Natural language processing is a subfield of AI.
  Score: 0.0000, Document: The quick brown fox jumps over the lazy dog.


In [24]:
# Perform BM25 search
print(f"BM25 search results for '{query_ai}':")
bm25_results = rag_system_st.bm25_search(query_ai)
for doc, score in bm25_results:
    print(f"  Score: {score:.4f}, Document: {doc}")

print(f"\nBM25 search results for '{query_space}':")
bm25_results = rag_system_st.bm25_search(query_space)
for doc, score in bm25_results:
    print(f"  Score: {score:.4f}, Document: {doc}")

BM25 search results for 'What is AI?':
  Score: 0.2319, Document: Artificial intelligence is a rapidly evolving field.
  Score: 0.2197, Document: Natural language processing is a subfield of AI.
  Score: 0.2087, Document: The moon is a natural satellite of the Earth.
  Score: 0.1815, Document: The sun is a star at the center of our solar system.
  Score: 0.0000, Document: The quick brown fox jumps over the lazy dog.

BM25 search results for 'Tell me about the moon.':
  Score: 0.2087, Document: The moon is a natural satellite of the Earth.
  Score: 0.2087, Document: The quick brown fox jumps over the lazy dog.
  Score: 0.1815, Document: The sun is a star at the center of our solar system.
  Score: 0.0000, Document: Natural language processing is a subfield of AI.
  Score: 0.0000, Document: Artificial intelligence is a rapidly evolving field.


In [25]:
# Perform hybrid search
print(f"Hybrid search results for '{query_ai}':")
hybrid_results = rag_system_st.hybrid_search(query_ai)
for doc, score in hybrid_results:
    print(f"  Score: {score:.4f}, Document: {doc}")

print(f"\nHybrid search results for '{query_space}':")
hybrid_results = rag_system_st.hybrid_search(query_space)
for doc, score in hybrid_results:
    print(f"  Score: {score:.4f}, Document: {doc}")

Hybrid search results for 'What is AI?':
  Score: 0.5000, Document: Artificial intelligence is a rapidly evolving field.
  Score: 0.4737, Document: Natural language processing is a subfield of AI.
  Score: 0.4500, Document: The moon is a natural satellite of the Earth.
  Score: 0.3913, Document: The sun is a star at the center of our solar system.
  Score: 0.0000, Document: The quick brown fox jumps over the lazy dog.

Hybrid search results for 'Tell me about the moon.':
  Score: 0.5000, Document: The quick brown fox jumps over the lazy dog.
  Score: 0.5000, Document: The moon is a natural satellite of the Earth.
  Score: 0.4348, Document: The sun is a star at the center of our solar system.
  Score: 0.0000, Document: Natural language processing is a subfield of AI.
  Score: 0.0000, Document: Artificial intelligence is a rapidly evolving field.
